# Installing the necessary libraries

In [5]:
!pip install numpy
!pip install torch
!pip install sklearn
!pip install pytorch_transformers
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# SciBERT

In [6]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("/Users/lfoppiano/development/projects/embeddings/pre-trained-embeddings/scibert/scibert_scivocab_cased_hf")
model = AutoModel.from_pretrained("/Users/lfoppiano/development/projects/embeddings/pre-trained-embeddings/scibert/scibert_scivocab_cased_hf",output_hidden_states=True)

Some weights of the model checkpoint at /Users/lfoppiano/development/projects/embeddings/pre-trained-embeddings/scibert/scibert_scivocab_cased_hf were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Vector calculation

Borrowed at https://github.com/stockmarkteam/bert-book/blob/master/Chapter10.ipynb
thanks to Oka-san

This method averages the last hidden state values, using the attention mask for coefficients


In [9]:
import numpy as np
import torch
import glob

def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
    return torch.max(token_embeddings, 1)[0]

max_length = 256
sentence_vectors = []
classifications = []
attention_masks = []
for file in glob.glob(f'./resources/supermat_classification.?.tsv'):
    print(file)
    lines = open(file).read().splitlines()
    for line in lines:
        split = line.split("\t")
        if len(split) < 2:
            print("skip ", line)
            continue

        text = split[1]
        classification = split[2]

        encoding = tokenizer(
            text,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        encoding = { k: v for k, v in encoding.items() }
        attention_mask = encoding['attention_mask']

        with torch.no_grad():
            output = model(**encoding)
            # last_hidden_state = output.last_hidden_state
            # cross_attention = output.cross_attentions
            # averaged_hidden_state = (last_hidden_state * attention_mask.unsqueeze(-1)).sum(1) / attention_mask.sum(1, keepdim=True)

        sentence_vectors = max_pooling(output, attention_mask)
        classifications.append(classification)


sentence_vectors = np.vstack(sentence_vectors)
labels = np.array(classifications)

./resources/supermat_classification.1.tsv
./resources/supermat_classification.0.tsv


## Dimension reduction


In [10]:
from sklearn.decomposition import PCA

sentence_vectors_pca = PCA(n_components=2).fit_transform(sentence_vectors)

ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

# plot

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20,20))
for label in range(2):
    plt.subplot(2,2,label+1)
    # index = [True if l == label else False for l in labels]
    index = str(label) == labels
    plt.plot(
        sentence_vectors_pca[:,0],
        sentence_vectors_pca[:,1],
        'o',
        markersize=1,
        color=[0.7, 0.7, 0.7]
    )
    plt.plot(
        sentence_vectors_pca[index,0],
        sentence_vectors_pca[index,1],
        'o',
        markersize=2,
        color='k'
    )
    plt.title(label)

In [ ]:
from sklearn.manifold import TSNE

sentence_vectors_tsne = TSNE(n_components=2).fit_transform(sentence_vectors)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20,20))
for label in range(2):
    plt.subplot(2,2,label+1)
    # index = [True if l == label else False for l in labels]
    index = str(label) == labels
    plt.plot(
        sentence_vectors_tsne[:,0],
        sentence_vectors_tsne[:,1],
        'o',
        markersize=1,
        color=[0.7, 0.7, 0.7]
    )
    plt.plot(
        sentence_vectors_tsne[index,0],
        sentence_vectors_tsne[index,1],
        'o',
        markersize=2,
        color='k'
    )
    plt.title(labels[label])